In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import os
for dirname, _, filenames in os.walk('../../data/lyrics'):
    for filename in filenames:
        print(os.path.join(dirname, filename).replace("\\","/"))

print(50*'-')

for dirname, _, filenames in os.walk('../../data/billboard'):
    for filename in filenames:
        print(os.path.join(dirname, filename).replace("\\","/"))

from src.progress_bar import printProgressBar

../../data/lyrics/artist_song_lyrics.csv
../../data/lyrics/bb-t100-lyrics.csv
../../data/lyrics/bb_t100_lyrics_en.csv
../../data/lyrics/corona-lyrics.csv
../../data/lyrics/lyrics_invalid.json
../../data/lyrics/lyrics_invalid_updated.csv
../../data/lyrics/backups/artist_song_lyrics_bak.csv
--------------------------------------------------
../../data/billboard/bb_t100_en.csv
../../data/billboard/billboard.csv
../../data/billboard/billboard_top_100_2020.csv
../../data/billboard/backups/bb-t100_backup.csv


In [2]:
corona_lines = pd.read_excel('../../data/input/corona_lines.xlsx')
corona_lines

,Unnamed: 0,line,artist,song,corona
0,0,thank you's don't cut it,Adam Hambrick,Between Me and the End of the World,False
1,1,i mean what do you get,Adam Hambrick,Between Me and the End of the World,False
2,2,for the one who sees a crowd running up,Adam Hambrick,Between Me and the End of the World,False
3,3,and then decides to run in,Adam Hambrick,Between Me and the End of the World,False
4,4,its grace under fire,Adam Hambrick,Between Me and the End of the World,False
...,...,...,...,...,...
1814,1814,"just like you wanna stick a dick up in me, i w...",21 Savage & Metro Boomin Featuring Drake,Mr. Right Now,False
1815,1815,and don't get in your motherfuckin' feelings w...,21 Savage & Metro Boomin Featuring Drake,Mr. Right Now,False
1816,1816,and one of your niggas wanna get down on me,21 Savage & Metro Boomin Featuring Drake,Mr. Right Now,False
1817,1817,and i wanna give them some pussy 'cause they g...,21 Savage & Metro Boomin Featuring Drake,Mr. Right Now,False


In [3]:
import spacy
from spacy.tokens import DocBin

In [4]:
corona_lines.columns

Index(['Unnamed: 0', 'line', 'artist', 'song', 'corona'], dtype='object')

In [5]:
columns = ['line', 'corona']
data = corona_lines[columns]
data = data.sample(frac=1).reset_index(drop=True)
train = valid = data

In [6]:
nlp = spacy.load("en_core_web_lg")

In [7]:
def make_docs(df):
    docs = []
    for row in df[['line', 'corona']].itertuples():
        line = row[1]
        corona = row[2]
        doc = nlp(line)
        if corona:
            doc.cats['is_corona_line'] = 1
            doc.cats['is_not_corona_line'] = 0
        if not corona:
            doc.cats['is_corona_line'] = 0
            doc.cats['is_not_corona_line'] = 1
        docs.append(doc)
    return docs

In [8]:
train_docs = make_docs(train)
DocBin(docs=train_docs).to_disk('../../spacy/train.spacy')

valid_docs = make_docs(train)
DocBin(docs=valid_docs).to_disk('../../spacy/valid.spacy')


In [9]:
train.to_csv('../../spacy/train.csv', encoding='utf-8')
valid.to_csv('../../spacy/valid.csv', encoding='utf-8')